In [1]:
import os
os.getcwd()

'/home/ubuntu/NCI/notebooks'

In [2]:
import os
os.chdir("/home/ubuntu/NCI")
os.getcwd()

'/home/ubuntu/NCI'

In [3]:
import json
import numpy as np
import pandas as pd


pd.set_option("display.precision", 4)
pd.options.display.float_format = '{:,.4f}'.format

In [4]:
def load_coherence_test_result(dataset_name, base_folder, model_name, exp_suffix="", verbose=False):
    mrr = []
    test_config = json.load(open(f"./config/{dataset_name}_coherence_test.json", "r"))
    for data_suffix in test_config["semantic_relation"]:
        for model_suffix in test_config["semantic_relation"][data_suffix]:
            if model_suffix[:-1]=="random" or model_suffix=="common_neighbor":
                continue
            data_df = pd.DataFrame()
            for similarity in test_config["semantic_relation"][data_suffix]:
                df = pd.read_csv(f"{base_folder}/{dataset_name}/exp/{similarity}_exp_{dataset_name}_{data_suffix}_{model_name}_{model_suffix}{exp_suffix}.csv", index_col=0)
                df["Encoder"] = [f"{model_name}_{model_suffix}"] * len(df)
                df["Relation"] = [similarity] * len(df)
                data_df = pd.concat([data_df, df]) 
            data_df = data_df.reset_index(drop=True)
            data_df["Coherence Rate"] = data_df[["Group Coherence Rate", "Rank Coherence Rate"]].mean(axis=1)
            result_df = data_df.groupby(by="Relation")[["Coherence Rate"]].mean().reset_index()
            conditions = (
                (result_df["Relation"]!="random1")
                & (result_df["Relation"]!="random2")
                & (result_df["Relation"]!="random3")
                & (result_df["Relation"]!="random4")
                & (result_df["Relation"]!="random5")
                & (result_df["Relation"]!="common_neighbor")
            )
            result_df = result_df[conditions].reset_index(drop=True)
            result_df["rank"] = result_df["Coherence Rate"].rank(ascending=False)
            rel_rank = result_df[result_df["Relation"]==model_suffix]["rank"].item()
            mrr.append(1/rel_rank)

        print(f"{dataset_name} Accuracy: {np.array(mrr).mean():.3f}")
    return np.array(mrr).mean()


# Accuracy

In [5]:
task = "exp"
base_folder = f"./data"
model_name = "svd"
data_suffix = "baseline"
datasets = {
    'Cora': 'Cora',
}

In [6]:
scores = []
methods = []
data_names = []
exp_suffix =512 
for dataset_name in datasets:
    coherence_mrr = load_coherence_test_result(dataset_name, base_folder, model_name, f"_{exp_suffix}", verbose=True)
    scores.append(coherence_mrr)
    methods.append("Coherence Rate")
    data_names.append(dataset_name)

Cora Accuracy: 0.929
